In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd

import sys
sys.path.append("../")
import os

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.models import load_LM
import torch
from nnsight import LanguageModel
from src.utils import env_utils

# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

# lm = load_LM(
#     model_key=model_key,
#     torch_dtype=torch.float16,
#     load_in_4bit=True,
# )
lm = LanguageModel(
    os.path.join(env_utils.DEFAULT_MODELS_DIR, model_key), 
    device_map="auto", 
    load_in_4bit=True, 
    torch_dtype=torch.float16, 
    dispatch=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [01:15<00:00,  2.51s/it]


In [4]:
from src.dataset import load_worldstate_dataset
dataset = load_worldstate_dataset()
q, a = dataset.__getitem__(5, tags=["A", "B"])

print(q)
print(a)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose the correct option by predicting the answer option (either A or B) after the "Answer:" tag.

Story: Ravi is a farmer in a small Indian village, tending to his mango orchard. Ravi wants to harvest ripe mangoes to sell at the local market. He notices a mango tree with a heavy branch laden with ripe mangoes. A mischievous monkey comes along and eats all the ripe mangoes on the branch while Ravi is gathering his harvesting tools. Ravi sees the monkey eating the ripe mangoes.

Question: Does the branch have ripe mangoes or no ripe mangoes?
A) The branch has ripe mangoes.
B) The branch has no ripe mangoes.
Answer:
B


In [5]:
lm._model.dtype

torch.float16

In [6]:
from src.models import prepare_input
from src.functional import interpret_logits
from src.functional import free_gpu_cache

@torch.inference_mode()
def get_lm_answer(
    lm: LanguageModel,
    question: str, 
):
    free_gpu_cache()

    inputs = prepare_input(
        prompts = question,
        tokenizer= lm
    )
    with lm.trace(inputs, scan=False, validate=False) as tr:
        logits = lm.output.logits.save()

    predicted_token = interpret_logits(
        tokenizer=lm,
        logits = logits[:, -1],
        k = 10, 
        get_proba=True
    )[0]

    return predicted_token[0].strip()

get_lm_answer(lm, q)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


'B'

In [9]:
from tqdm.auto import tqdm
import random

tag_options = [
    ("a", "b"),
    ("x", "y"), 
    ("p", "q"),
]

progress = tqdm(range(len(dataset)))
correct = 0
for i in progress:
    question, answer = dataset.__getitem__(i, tags=random.choice(tag_options))
    lm_ans = get_lm_answer(lm=lm, question=question)
    correct += lm_ans.strip() == answer.strip()
    # print(f"expected: {answer.strip()} | got: {lm_ans.strip()}")
    progress.set_description(f"{i+1}/{len(dataset)} => acc: {correct/(i+1)}({correct=})")

200/200 => acc: 0.99(correct=198): 100%|██████████| 200/200 [10:01<00:00,  3.01s/it]              
